<div class="section" id="s-module-django.db.models">
<span id="s-models"></span><span id="module-django.db.models"></span><span id="models"></span><h1>Models<a class="headerlink" href="#module-django.db.models" title="Permalink to this headline">¶</a></h1>
<p>A model is the single, definitive source of information about your data. It
contains the essential fields and behaviors of the data you’re storing.
Generally, each model maps to a single database table.</p>
<p>The basics:</p>
<ul class="simple">
<li>Each model is a Python class that subclasses
<a class="reference internal" href="../../../ref/models/instances/#django.db.models.Model" title="django.db.models.Model"><code class="xref py py-class docutils literal"><span class="pre">django.db.models.Model</span></code></a>.</li>
<li>Each attribute of the model represents a database field.</li>
<li>With all of this, Django gives you an automatically-generated
database-access API; see <a class="reference internal" href="../queries/"><em>Making queries</em></a>.</li>
</ul>
<div class="section" id="s-quick-example">
<span id="quick-example"></span><h2>Quick example<a class="headerlink" href="#quick-example" title="Permalink to this headline">¶</a></h2>
<p>This example model defines a <code class="docutils literal"><span class="pre">Person</span></code>, which has a <code class="docutils literal"><span class="pre">first_name</span></code> and
<code class="docutils literal"><span class="pre">last_name</span></code>:</p>

```python
from django.db import models

class Person(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)

```

The above Person model would create a database table like this:

```sql
CREATE TABLE myapp_person (
    "id" serial NOT NULL PRIMARY KEY,
    "first_name" varchar(30) NOT NULL,
    "last_name" varchar(30) NOT NULL
);
```



The name of the table, myapp_person, is automatically derived from model metadata.
An id field is added automatically.
Django uses SQL tailored to the database backend specified in your settings file.


### Fields

In [ ]:
from django.db import models

class Musician(models.Model):
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
    instrument = models.CharField(max_length=100)

class Album(models.Model):
    artist = models.ForeignKey(Musician, on_delete=models.CASCADE)
    name = models.CharField(max_length=100)
    release_date = models.DateField()
    num_stars = models.IntegerField()

## Working with Models


In [ ]:
from django.db import models

class Address(models.Model):

    address = models.CharField(max_length=255, blank=True)
    city = models.CharField(max_length=150, blank=True)
    state = models.CharField(max_length=2, blank=True)
    zip = models.CharField(max_length=15, blank=True)

class Contact(models.Model):

    first_name = models.CharField(max_length=255, blank=True)
    last_name = models.CharField(max_length=255, blank=True)

    birthdate = models.DateField(auto_now_add=True)
    phone = models.CharField(max_length=25, blank=True)
    email = models.EMailField(blank=True)

    address = models.ForeignKey(Address, null=True)
    
    
nathan = Contact()
nathan.first_name = 'Nathan'
nathan.last_name = 'Yergler'
nathan.save()

** Starting in Django 1.5, calling .save() only updates the fields that have changed. **

## Managers

* Models get a manager injected as .objects
* Managers allow you to operate over collections of your model

```python
Contact.objects.filter(last_name__iexact='yergler')
Contact.objects.filter(address__state='OH')
```

## Custom Mnagers

```python
class ContactManager(models.Manager):

    def with_email(self):
        return self.filter(email__ne = '')

class Contact(models.Model):
    ...

    objects = ContactManager()
```

```python
contacts.objects.with_email().filter(email__endswith='osu.edu')
```

## Querying Your Data
```python
Contact.objects.filter(state='OH').filter(email__ne='')
```

* Query Sets are chainable
* Multiple filters are collapsed into SQL “and” conditions

#### OR conditions in Queries

If you need to do “or” conditions, you can use Q objects

```python
from django.db.models import Q

Contact.objects.filter(
    Q(state='OH') | Q(email__endswith='osu.edu')
)
```



## ORM Performance
#### Instantiation is Expensive

* QuerySets are lazy, but have non-trivial overhead when evaluated
* If a query returns 1000s of rows, users will notice this
* `.values()` and `.values_list()` avoid instantiation


#### Bad:

```python
for user in Users.objects.filter(is_active=True):
    send_email(user.email)
```


#### Good:

```python
user_emails = Users.objects.\
    filter(is_active=True).\
    values_list('email', flat=True)

for email in user_emails:
    send_email(email)
```


#### Traversing Relationships


* Traversing foreign keys can incur additional queries
* select_related queries for foreign keys in the initial query

```python
Contact.objects.\
    select_related('address').\
    filter(last_name = 'Yergler')
```



#### Query Performance

* If possible, don’t chain more than one filter

Because querysetsmaintain a lot of state and they get cloned during chaining

#### Use Raw SQL

* Sometimes the best option.
```python
Contact.objects.raw('SELECT * FROM contacts WHERE last_name = %s', [lname])
```

* Of course, DO NOT use string formatting in raw() calls

## Many-to-many relationships

In [ ]:
from django.db import models

class Publication(models.Model):
    title = models.CharField(max_length=30)

    def __str__(self):              # __unicode__ on Python 2
        return self.title

    class Meta:
        ordering = ('title',)

class Article(models.Model):
    headline = models.CharField(max_length=100)
    publications = models.ManyToManyField(Publication) # ManyToManyField

    def __str__(self):              # __unicode__ on Python 2
        return self.headline

    class Meta:
        ordering = ('headline',)

#### Create a couple of Publications:

```python
>>> p1 = Publication(title='The Python Journal')
>>> p1.save()
>>> p2 = Publication(title='Science News')
>>> p2.save()
>>> p3 = Publication(title='Science Weekly')
>>> p3.save()
```



#### Create an article 

```python
>>> a1 = Article(headline='Django lets you build Web apps easily')
>>> a1.save() # cannot associate without saving. Why?
```

```python
>>> a1.publications.add(p1)
>>> a2 = Article(headline='NASA uses Python')
>>> a2.save()
>>> a2.publications.add(p1, p2)
>>> a2.publications.add(p3)
```

Many-to-many relationships can be queried using lookups across relationships:
```python
>>> Article.objects.filter(publications__id=1)
[<Article: Django lets you build Web apps easily>, <Article: NASA uses Python>]
>>> Article.objects.filter(publications__pk=1)
[<Article: Django lets you build Web apps easily>, <Article: NASA uses Python>]
>>> Article.objects.filter(publications=1)
[<Article: Django lets you build Web apps easily>, <Article: NASA uses Python>]
>>> Article.objects.filter(publications=p1)
[<Article: Django lets you build Web apps easily>, <Article: NASA uses Python>]

>>> Article.objects.filter(publications__title__startswith="Science")
[<Article: NASA uses Python>, <Article: NASA uses Python>]

>>> Article.objects.filter(publications__title__startswith="Science").distinct()
[<Article: NASA uses Python>]
```


Reverse m2m queries are supported (i.e., starting at the table that doesn’t have a ManyToManyField):

```python
>>> Publication.objects.filter(id=1)
[<Publication: The Python Journal>]
>>> Publication.objects.filter(pk=1)
[<Publication: The Python Journal>]

>>> Publication.objects.filter(article__headline__startswith="NASA")
[<Publication: Highlights for Children>, <Publication: Science News>, <Publication: Science Weekly>, <Publication: The Python Journal>]

>>> Publication.objects.filter(article__id=1)
[<Publication: The Python Journal>]
>>> Publication.objects.filter(article__pk=1)
[<Publication: The Python Journal>]
>>> Publication.objects.filter(article=1)
[<Publication: The Python Journal>]
>>> Publication.objects.filter(article=a1)
[<Publication: The Python Journal>]

>>> Publication.objects.filter(article__in=[1,2]).distinct()
[<Publication: Highlights for Children>, <Publication: Science News>, <Publication: Science Weekly>, <Publication: The Python Journal>]
>>> Publication.objects.filter(article__in=[a1,a2]).distinct()
[<Publication: Highlights for Children>, <Publication: Science News>, <Publication: Science Weekly>, <Publication: The Python Journal>]

```

## Many-to-one relationships

In [ ]:
rom django.db import models

class Reporter(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)
    email = models.EmailField()

    def __str__(self):              # __unicode__ on Python 2
        return "%s %s" % (self.first_name, self.last_name)

class Article(models.Model):
    headline = models.CharField(max_length=100)
    pub_date = models.DateField()
    reporter = models.ForeignKey(Reporter, on_delete=models.CASCADE) #ForeignKey

    def __str__(self):              # __unicode__ on Python 2
        return self.headline

    class Meta:
        ordering = ('headline',)


```python
>>> r = Reporter(first_name='John', last_name='Smith', email='john@example.com')
>>> r.save()

>>> r2 = Reporter(first_name='Paul', last_name='Jones', email='paul@example.com')
>>> r2.save()

>>> from datetime import date
>>> a = Article(id=None, headline="This is a test", pub_date=date(2005, 7, 27), reporter=r)
>>> a.save()

>>> a.reporter.id
1

>>> a.reporter
<Reporter: John Smith>


# Find all Articles for any Reporter whose first name is "John".
>>> Article.objects.filter(reporter__first_name='John')
[<Article: John's second story>, <Article: This is a test>]


```

## One-to-one relationships

In [ ]:
from django.db import models

class Place(models.Model):
    name = models.CharField(max_length=50)
    address = models.CharField(max_length=80)

    def __str__(self):              # __unicode__ on Python 2
        return "%s the place" % self.name

class Restaurant(models.Model): #OneToOneField
    place = models.OneToOneField(
        Place,
        on_delete=models.CASCADE,
        primary_key=True,
    )
    serves_hot_dogs = models.BooleanField(default=False)
    serves_pizza = models.BooleanField(default=False)

    def __str__(self):              # __unicode__ on Python 2
        return "%s the restaurant" % self.place.name

class Waiter(models.Model):
    restaurant = models.ForeignKey(Restaurant, on_delete=models.CASCADE)
    name = models.CharField(max_length=50)

    def __str__(self):              # __unicode__ on Python 2
        return "%s the waiter at %s" % (self.name, self.restaurant)

```python
>> p1 = Place(name='Demon Dogs', address='944 W. Fullerton')
>>> p1.save()
>>> p2 = Place(name='Ace Hardware', address='1013 N. Ashland')
>>> p2.save()
>>> r = Restaurant(place=p1, serves_hot_dogs=True, serves_pizza=False)
>>> r.save()


>>> r.place
<Place: Demon Dogs the place>



>>> p1.restaurant
<Restaurant: Demon Dogs the restaurant>

```

You can query the models using lookups across relationships:

```python
>>> Restaurant.objects.get(place=p1)
<Restaurant: Demon Dogs the restaurant>
>>> Restaurant.objects.get(place__pk=1)
<Restaurant: Demon Dogs the restaurant>
>>> Restaurant.objects.filter(place__name__startswith="Demon")
[<Restaurant: Demon Dogs the restaurant>]
>>> Restaurant.objects.exclude(place__address__contains="Ashland")
[<Restaurant: Demon Dogs the restaurant>]
```
